In [50]:
import pandas as pd
import numpy as np
import json
import codecs

In [2]:
from tqdm import tqdm

In [3]:
import random

In [4]:
from src.refine_text import normalize_golden_text

In [5]:
file_path = '/Users/bryan/Downloads/vinmec_report_80k_unique_lines.txt'

In [6]:
with open(file_path, 'rt') as f:
    lines = [line.rstrip() for line in f]

In [7]:
gold_data_refined = []
gold_data_raw = []

for line in tqdm(lines):
    try:
        content = normalize_golden_text(line)
    except:
        print(line)
        break
    if len(content.split()) > 5:
        gold_data_refined.append(content)
        gold_data_raw.append(line)

100%|██████████| 79369/79369 [00:25<00:00, 3118.73it/s]


In [14]:
export_data = pd.DataFrame(zip(gold_data_refined, gold_data_raw), columns=['refined', 'raw'])

In [15]:
export_data.to_csv('/Users/bryan/Desktop/export_data_common_voices.csv', encoding='UTF-8')

In [16]:
from string import Template

In [17]:
example = Template('$verb $sentence_loc $prep $what')

In [18]:
example.substitute(verb='chỉnh', sentence_loc='câu 7', prep='thành', what='tim to')

'chỉnh câu 7 thành tim to'

In [77]:
templates = {
    'modify': {
        'sentences': [
            '$verb $sentence_loc $prep $what'
        ],
        'verb': [
            'chỉnh', 'sửa', 'thay đổi'
        ],
        'sentence_loc': [
            'câu $number'
        ],
        'prep': [
            'thành', 'bằng', 'về', ''
        ]
    },
    'delete': {
        'sentences': [
            '$verb $sentence_loc'
        ],
        'sentence_loc': [
            'câu $number'
        ],
        'verb': [
            'xóa', 'bỏ', 'loại'
        ]
    },
    'add': {
        'sentences': [
            '$verb $prep $sentence_loc $what',
            '$verb $what $prep $sentence_loc'
        ],
        'verb': [
            'thêm'
        ],
        'prep': [
            'vào trước', 'vào sau', 'vào sau'
        ],
        'sentence_loc': [
            'câu $number'
        ]
    }
}

In [78]:
with open('/Users/bryan/Desktop/template.json', 'w') as outfile:
    json.dump(templates, outfile)

In [20]:
prob_add, prob_del, prob_modify = random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)

In [21]:
ind_case = np.argmax([prob_add, prob_del, prob_modify])
if ind_case == 0:
    case = 'modify'
elif ind_case == 1:
    case = 'delete'
else:
    case = 'add'
case

'modify'

In [94]:
number = random.randint(1, 10)
example = random.sample(templates[case]['sentences'], 1)[0]
verb = random.sample(templates[case]['verb'], 1)[0]
prep = random.sample(templates[case]['prep'], 1)[0]
what = random.sample(gold_data_refined, 1)[0]
sentence_loc = random.sample(templates[case]['sentence_loc'], 1)[0]

In [95]:
example, verb, prep, what, sentence_loc

('$verb $sentence_loc $prep $what',
 'thay đổi',
 'về',
 'không thấy tiêu xương phá vỡ vỏ xương hay xâm lấn phần mềm',
 'câu $number')

In [96]:
example = Template(example)
sentence_loc = Template(sentence_loc)
sentence_loc = sentence_loc.substitute(number=number)
text = example.substitute({'verb': verb, 'sentence_loc': sentence_loc, 'prep': prep, 'what': what})

In [97]:
text

'thay đổi câu 4 về không thấy tiêu xương phá vỡ vỏ xương hay xâm lấn phần mềm'

In [83]:
def generate_text_voice(templates, finding:str) -> str:
    prob_add, prob_del, prob_modify, prob_nothing = random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)
    ind_case = np.argmax([prob_add, prob_del, prob_modify, prob_nothing])
    if ind_case == 0:
        case = 'modify'
    elif ind_case == 1:
        case = 'delete'
    elif ind_case == 2:
        case = 'add'
    else:
        case = 'nothing'
    if templates.get(case, None) is not None:
        if templates[case].get('sentences') is not None:
            example = random.sample(templates[case]['sentences'], 1)[0]
        else:
            print(f'case {case} has error on sentences')
            
        if templates[case].get('verb') is not None:
            verb = random.sample(templates[case]['verb'], 1)[0]
        else:
            print(f'case {case} has error on verb')
            
        if templates[case].get('prep') is not None:
            prep = random.sample(templates[case]['prep'], 1)[0]
        else:
            print(f'case {case} has error on prep')
        
        if templates[case].get('sentence_loc') is not None:
            sentence_loc = random.sample(templates[case]['sentence_loc'], 1)[0]
        else:
            print(f'case {case} has error on sentence_loc')
        
        number = random.randint(1, 10)
        example = Template(example)
        sentence_loc = Template(sentence_loc)
        sentence_loc = sentence_loc.substitute(number=number)
        text = example.substitute(verb=verb, sentence_loc=sentence_loc, prep=prep, what=finding)
        return text, case, number
    else:
        return finding, None, None

In [46]:
generate_text_voice(templates, random.sample(gold_data_refined, 1)[0])

('thêm vào sau câu 10 finding normal heart shadow and mediastinum', 'add', 10)

In [79]:
templates = json.loads(codecs.open('/Users/bryan/Desktop/template.json', 'r', 'UTF-8').read())

In [80]:
templates

{'modify': {'sentences': ['$verb $sentence_loc $prep $what'],
  'verb': ['chỉnh', 'sửa', 'thay đổi'],
  'sentence_loc': ['câu $number'],
  'prep': ['thành', 'bằng', 'về', '']},
 'delete': {'sentences': ['$verb $sentence_loc'],
  'sentence_loc': ['câu $number'],
  'verb': ['xóa', 'bỏ', 'loại']},
 'add': {'sentences': ['$verb $prep $sentence_loc $what',
   '$verb $what $prep $sentence_loc'],
  'verb': ['thêm'],
  'prep': ['vào trước', 'vào sau', 'vào sau'],
  'sentence_loc': ['câu $number']}}

In [54]:
generate_text_voice(templates, random.sample(gold_data_refined, 1)[0])

('thay đổi trước câu số 1 thành phổi phải xẹp gần toàn bộ', 'modify', 1)

In [57]:
foo = pd.read_excel('/Users/bryan/Downloads/Common Voice List_Vietnamese_100.xlsx')

In [84]:
def run(template_path, data_path, n_samples):
    templates = json.loads(codecs.open(template_path, 'r', 'UTF-8').read())
    database = pd.read_excel(data_path)
    database = database['refined'].tolist()
    records = []
    for finding in tqdm(database):
        tmp = []
        for j in range(n_samples):
            rec_ = generate_text_voice(templates, finding)
            tmp.append(rec_)
        tmp = list(set(tmp))
        records = records + tmp
    return records

In [85]:
bar = run('/Users/bryan/Desktop/template.json', '/Users/bryan/Downloads/Common Voice List_Vietnamese_100.xlsx', 2)

  2%|▏         | 2/101 [00:00<00:00, 3583.34it/s]

case delete has error on prep


UnboundLocalError: local variable 'prep' referenced before assignment

In [87]:
templates['delete'].keys()

dict_keys(['sentences', 'sentence_loc', 'verb'])

In [91]:
example.substitute??

Signature: example.substitute(*args, **kws)
Docstring: <no docstring>
Source:   
    def substitute(*args, **kws):
        if not args:
            raise TypeError("descriptor 'substitute' of 'Template' object "
                            "needs an argument")
        self, *args = args  # allow the "self" keyword be passed
        if len(args) > 1:
            raise TypeError('Too many positional arguments')
        if not args:
            mapping = kws
        elif kws:
            mapping = _ChainMap(kws, args[0])
        else:
            mapping = args[0]
        # Helper function for .sub()
        def convert(mo):
            # Check the most common path first.
            named = mo.group('named') or mo.group('braced')
            if named is not None:
                return str(mapping[named])
            if mo.group('escaped') is not None:
                return self.delimiter
            if mo.group('invalid') is not None:
                self._invalid(mo)
            raise